<a href="https://colab.research.google.com/github/xcellentbird/TOY-PROJECTS/blob/main/png_jpg_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import PIL
import torchvision
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision.utils import save_image
from torchvision import transforms

!pip install timm
import timm

     |████████████████████████████████| 376 kB 7.1 MB/s 


In [ ]:
from google import colab

colab.files.upload()

Saving CIFAR10.zip to CIFAR10.zip


In [ ]:
import zipfile

zipfile.ZipFile('CIFAR10.zip').extractall('./')

In [ ]:
path = './CIFAR10'

# CIFAR10 데이터셋 format변환 및 다운로드

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset)
test_dataloader = torch.utils.data.DataLoader(test_dataset)

In [ ]:
for c in dataset.classes:
    os.mkdir(os.path.join(path,'jpg','train',c))
    os.mkdir(os.path.join(path,'jpg','test',c))
    os.mkdir(os.path.join(path,'png','train',c))
    os.mkdir(os.path.join(path,'png','test',c))

In [ ]:
for i, (img, label) in enumerate(train_dataloader):
    save_image(img, os.path.join(path, 'png', 'train', train_dataset.classes[label], '{:4}'.format(str(i)) + '.png'))
    save_image(img, os.path.join(path, 'jpg', 'train', train_dataset.classes[label], '{:4}'.format(str(i)) + '.png'))

for i, (img, label) in enumerate(test_dataloader):
    save_image(img, os.path.join(path, 'jpg', 'test', test_dataset.classes[label], '{:4}'.format(str(i)) + '.png'))
    save_image(img, os.path.join(path, 'png', 'test', test_dataset.classes[label], '{:4}'.format(str(i)) + '.png'))

--- --- ---

# png, jpg 데이터셋과 모델 준비

In [ ]:
transform = transforms.ToTensor()

In [ ]:
jpg_train_dataset = torchvision.datasets.ImageFolder(os.path.join(path, 'jpg','train'), transform=transform)
jpg_test_dataset = torchvision.datasets.ImageFolder(os.path.join(path, 'jpg','test'), transform=transform)

png_train_dataset = torchvision.datasets.ImageFolder(os.path.join(path, 'png','train'), transform=transform)
png_test_dataset = torchvision.datasets.ImageFolder(os.path.join(path, 'png','test'), transform=transform)

In [ ]:
jpg_train_dataloader = torch.utils.data.DataLoader(dataset=jpg_train_dataset, batch_size=16, shuffle=True)
jpg_test_dataloader = torch.utils.data.DataLoader(dataset=jpg_test_dataset, batch_size=16, shuffle=True)

png_train_dataloader = torch.utils.data.DataLoader(dataset=png_train_dataset, batch_size=16, shuffle=True)
png_test_dataloader = torch.utils.data.DataLoader(dataset=png_test_dataset, batch_size=16, shuffle=True)

dataloaders = {'jpg':[jpg_train_dataloader, jpg_test_dataloader], 'png':[png_train_dataloader, png_test_dataloader]}

In [ ]:
model_for_jpg = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(jpg_train_dataset.classes))
model_for_png = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(jpg_train_dataset.classes))

models = {"jpg":model_for_jpg, "png":model_for_png}

In [ ]:
for m in models:
  models[m] = nn.Sequential(models[m], nn.ReLU(), nn.Softmax(dim=-1))
  models[m] = models[m].cuda()

In [ ]:
freeze = False
if freeze:
    for m in models:
        for p in models[m].parameters():
            p.requires_grad = False

        for p in models[m].classifier.parameters():
            p.requires_grad = True
            

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
lr = 0.001
if freeze:
    optimizer_for_jpg = optim.SGD(models['jpg'].classifier.parameters(), lr=lr, momentum=0.5)
    optimizer_for_png = optim.SGD(models['png'].classifier.parameters(), lr=lr, momentum=0.5)
else:
    optimizer_for_jpg = optim.SGD(models['jpg'].parameters(), lr=lr, momentum=0.5)
    optimizer_for_png = optim.SGD(models['png'].parameters(), lr=lr, momentum=0.5)

optimizers = {'jpg':optimizer_for_jpg, 'png':optimizer_for_png}


CELoss = nn.CrossEntropyLoss().cuda()

In [ ]:
def train(model, train_loader, loss_function, optimizer, epoch):
  model.train()
  for idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    data, target = data.cuda(), target.cuda()
    output = model(data)

    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()

    if idx and not idx % 10:
      print('\rTrain Epoch: {} [{}/{} ({:.0f}%)] \t Loss: {:.6f}'.format(
        epoch, idx * len(data), len(train_loader.dataset),
        100. * idx / len(train_loader), loss.item()
      ), end='')

In [ ]:
def evaluate(model, test_loader, loss_function):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)

            test_loss += loss_function(output, target)

            output = output.cpu().numpy()
            
            pred = np.argmax(output, axis=1)
            #_, pred = output.max(dim=1, keepdim=True)
            correct += sum(target.cpu().numpy() == pred) #pred.eq(target.view_as(pred)).sum().item()
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
model = models['png']
for epoch in range(50):
  # train with png dataset
  train(model, dataloaders['png'][0], CELoss, optimizers['png'], epoch+1)

  # test with png dataset
  test_loss, test_accuracy = evaluate(model, dataloaders['png'][1], CELoss)
  print('\n[{}] png - Test Loss: {:.4f}, Accuracy: {:.8f}%'.format(
    epoch+1, test_loss, test_accuracy
  ))

  # test with jpg dataset
  test_loss, test_accuracy = evaluate(model, dataloaders['jpg'][1], CELoss)
  print('[{}] jpg - Test Loss: {:.4f}, Accuracy: {:.8f}%'.format(
    epoch+1, test_loss, test_accuracy
  ))

Train Epoch: 1 [49920/50000 (100%)] 	 Loss: 1.928047
[1] png - Test Loss: 1203.0369, Accuracy: 53.34000000%
[1] jpg - Test Loss: 1203.0367, Accuracy: 53.34000000%
Train Epoch: 2 [49920/50000 (100%)] 	 Loss: 1.979603
[2] png - Test Loss: 1183.8125, Accuracy: 56.66000000%
[2] jpg - Test Loss: 1183.8126, Accuracy: 56.66000000%
Train Epoch: 3 [49920/50000 (100%)] 	 Loss: 1.951107
[3] png - Test Loss: 1164.3923, Accuracy: 59.73000000%
[3] jpg - Test Loss: 1164.3926, Accuracy: 59.73000000%
Train Epoch: 4 [49920/50000 (100%)] 	 Loss: 1.804829
[4] png - Test Loss: 1151.6798, Accuracy: 61.61000000%
[4] jpg - Test Loss: 1151.6802, Accuracy: 61.61000000%
Train Epoch: 5 [49920/50000 (100%)] 	 Loss: 1.939555
[5] png - Test Loss: 1141.7509, Accuracy: 63.33000000%
[5] jpg - Test Loss: 1141.7502, Accuracy: 63.33000000%
Train Epoch: 6 [49920/50000 (100%)] 	 Loss: 1.691371
[6] png - Test Loss: 1132.0271, Accuracy: 64.90000000%
[6] jpg - Test Loss: 1132.0275, Accuracy: 64.90000000%
Train Epoch: 7 [49920/